## Spatial Modeling and Analytics
# Exploration
### A simple site selection example

In [ ]:
library(IRdisplay)

display_html(paste('<script type="text/javascript" src="../../supplementary/js/custom.js"></script><input id="toggle_code" type="button" value="Toggle raw code">'))


This exploration is the same workflow as one you may have already completed in the Beginner Spatial Modeling and Analytics lesson, but this time we're doing it using R. 

You may be interested to know that this translation from Python to R was completed with the assistance of ChatGPT! It wasn't perfect and needed some understanding of R to fix the bugs, but it did make the work faster. 

To continue, click the X in the top right to drop into the Notebook and continue reading and clicking on the run icon beside each code chunk. 

# Let's Create a Basic Site Suitability Model

## Goal: 
Find buildings in a city that are suitable candidates for a new coffee shop business.

## Criteria:
The candidate buildings should be:
1. A building type of commercial, retail, or office building
1. At least 400 meters from other coffee shops
1. Close to a bikepath
1. Close to a cinema

## The process:
1. Determine the criteria (done!)
1. Get data
1. Create buffers
1. Assign weights
1. Intersect and sum values

The result is a map showing the site suitability values. Suitability is indicated by the value - high values are highly suitable. 

# Load the packages
First, as usual, we need to import the appropriate R packages. The most important package we'll load is `sf`, the simple features package that provides a means of storing and retrieving spatial data. The data we're going to use comes from Open Street Map, so we need the package `osmdata`. 

In [ ]:
library(osmdata)
library(sf)
library(ggplot2)
library(gridExtra)
library(leaflet)

# Get the data

An OSMdata query begins by specifying a bounding box with the function `opq()`, followed by specifying the desired OSM features with `add_osm_feature()`.

Fortunately, you don't actually need to know the coordinates of the bounding box around your place of interest. You can simply use standard place names. If you want to run this notebook later for a different place, you can put a new placename in here. Remember that since OSM is crowd-sourced, you might not find all the places you want to use in the dataset. However, all major US and global cities are probably there. 

Our criteria require that we get data about coffee shops, bikepaths, cinemas and buildings. The OSM data contains all these kinds of data, but we have to extract each one separately for our model.

In this first block, we'll get features that are coffee shops in Minneapolis, Minnesota. 

We use OSMdata to create a spdf (spatial data frame) which is stored in the `coffee_points` variable.

In [ ]:
# Create a query for OSMdata with the bounding box and the features we wish to retrieve
query1 <- opq('minneapolis, MN')  %>% add_osm_feature(key = 'cuisine', value = 'coffee_shop')

# Create an OSMdata object using the query
coffee_shops <- osmdata_sf(query1)

# Create a spatial points ibject by selecting the point objects 
# in the OSM query result
coffee_points <- coffee_shops$osm_points

# Plot them!
ggplot() +
  geom_sf(data = coffee_points$geometry) +
  theme_minimal()

OK, how many did you find?

In [ ]:
nrow(coffee_points)

We got {{nrow(coffee_points)}} for now, but this number may change as the OSM data is constantly updated.

Now, before we move on to the next set of data, we need to do a coordinate transformation from lat/long to UTM so we can use cartesian geometry to measure distances. This was explained in the Beginner Spatial Modeling and Analytics lesson, check it out if you want to know more about this step. In any case, run this code below and move on. 

In [ ]:
# Change the CRS of coffee_points to UTM
coffee_points_UTM <- st_transform(coffee_points, "+proj=utm +zone=15 +datum=WGS84")


Next we'll get the bikepaths, aka in OSMdata as cycleways. 

In [ ]:
query2 <- opq('minneapolis, MN') %>%
  add_osm_feature(key = 'highway', value = 'cycleway')
cycleways <- osmdata_sf(query2)
cycleways_lines <- cycleways$osm_lines
ggplot() +
  geom_sf(data = cycleways_lines$geometry) +
  theme_minimal()
cycleways_lines_UTM <- st_transform(cycleways_lines, "+proj=utm +zone=15 +datum=WGS84")


Now get and transform the cinerma point features.

In [ ]:
query3 <- opq('minneapolis, MN') %>%
  add_osm_feature(key = 'amenity', value = 'cinema')
cinemas <- osmdata_sf(query3)
cinemas_points <- cinemas$osm_points
ggplot() +
  geom_sf(data = cinemas_points$geometry) +
  theme_minimal()
cinemas_points_UTM <- st_transform(cinemas_points, "+proj=utm +zone=15 +datum=WGS84")


Finally fetch the footprints (outlines, i.e. polygons) for commerical, retail and office buildings in Minneapolis. This may take some time, so be patient while waiting for the asterisk to change to a number.

In [ ]:
query4 <- opq('minneapolis, MN') %>%
  add_osm_features (features = list (
    "building" = "commercial",
    "building" = "retail",
    "building" = "offices"
  ))

buildings <- osmdata_sf(query4)
buildings_polygons <- buildings$osm_polygons
ggplot() +
  geom_sf(data = buildings_polygons$geometry) +
  theme_minimal()
buildings_polygons_UTM <- st_transform(buildings_polygons, "+proj=utm +zone=15 +datum=WGS84")


Now, we have all our data. Let's go back to the criteria so we can see how we need to manipulate these data.

Recall that the candidate buildings should be:

1. A building type of commercial, retail, or office building
1. At least 400 meters from other coffee shops
1. Close to a bikepath
1. Close to a cinema

OK, we've already taken care of criteria #1 by getting data about only buildings of these types. To do criteria #2 we need to create buffers...

## Create buffers

Buffers are used to define the area of influence of features. We'll buffer the coffee shops by 400m as an exclusion zone in which we don't want to select candidate sites. 

In [ ]:
buffered_coffee <- st_buffer(coffee_points_UTM, dist = 400)

ggplot() +
  geom_sf(data = coffee_points_UTM, color = "blue", alpha = 0.5) +
  geom_sf(data = buffered_coffee, color = "red", fill = "transparent") +
  coord_sf(crs = st_crs(coffee_points_UTM))

Good. This plot shows all those areas that are within 400m buffer of existing coffee shops. We do not want to include buildings in these areas in our result. 

Now we need to deal with the final two criteria in which locations close to cinemas and bikepaths are more favorable than those that are farther away. Thus places nearby should have higher value in our site selection than places far away - we do this by assigning weights.

## Assign weights

There are many ways to assign weights in site suitability models. Since this is all vector data, we're going to assign weights by creating concentric buffers with declining value as distance from the feature increases. For example, we prefer places that are close to cinemas, so locations that are less than 500 m get a higher weight than places between 500 and 1000 m, and those get more than places 1000 to 1500m away. Anything futher than 1500 gets no weight at all! 

Let's see how this works with our Cinema data.

In [ ]:
cinemas500 <- st_buffer(cinemas_points_UTM, dist = 500)
cinemas1000 <- st_buffer(cinemas_points_UTM, dist = 1000)
cinemas1500 <- st_buffer(cinemas_points_UTM, dist = 1500)

ggplot() +
  geom_sf(data = cinemas_points_UTM, color = "black", alpha = 0.5) +
  geom_sf(data = cinemas500, color = "red", fill = "transparent") +
  geom_sf(data = cinemas1000, color = "blue", fill = "transparent") +
  geom_sf(data = cinemas1500, color = "green", fill = "transparent") +
  coord_sf(crs = st_crs(cinemas_points_UTM))  

Note how the buffers nest inside of each other and some of them overlap. so next we need to perform a union operation to comine them. 

In [ ]:
cinemas500u <- st_union(cinemas500)
cinemas1000u <- st_union(cinemas1000)
cinemas1500u <- st_union(cinemas1500)

ggplot() +
  geom_sf(data = cinemas_points_UTM, color = "black", alpha = 0.5) +
  geom_sf(data = cinemas500u, color = "red", fill = "transparent") +
  geom_sf(data = cinemas1000u, color = "blue", fill = "transparent") +
  geom_sf(data = cinemas1500u, color = "green", fill = "transparent") +
  coord_sf(crs = st_crs(cinemas_points_UTM))  

And now we need to difference them to make concentric bands. There's a lot of code below that's needed to make sure that the data is in the correct format. Feel free to just click run through the next code cell, watching the plot to see how the bands evolve. 

In [ ]:
cinemas3 <- st_sf(cinemas500u)
plot1 <- ggplot() +
  geom_sf(data = cinemas3, fill = "green")+
  labs(title = "Cinemas 500u")

cinemas2 <- st_difference(cinemas1000u, cinemas500u)
cinemas2 <- st_sf(cinemas2)
plot2 <- ggplot() +
  geom_sf(data = cinemas2, fill = "blue") +
  labs(title = "Diff btw 1000 & 500")

cinemas1 <- st_difference(cinemas1500u, cinemas1000u)
cinemas1 <- st_sf(cinemas1)
plot3 <- ggplot() +
  geom_sf(data = cinemas1, fill = "red") +
  labs(title = "Diff btw 1500 & 1000")

plot4 <- ggplot() +
  geom_sf(data = cinemas3, fill = "green") +
  geom_sf(data = cinemas2, fill = "blue") +
  geom_sf(data = cinemas1, fill = "red") +
  coord_sf(crs = st_crs(cinemas_points_UTM))  +
  labs(title = "Union")

grid.arrange(plot1, plot2, ncol = 2)
grid.arrange(plot3, plot4, ncol = 2)

Wasn't that cool? 

Now we'll assign weights to the concentric circles. Weights are 3 for the the inside circle, 2 for the middle band and 1 for the furthest away. Then we combine them into a single spatial polygon object. Once again, there's extra code to make sure the spatial objects have geometry and the right labels so they can be combined.

In [ ]:
cinemas3$weight <- 3
cinemas2$weight <- 2
cinemas1$weight <- 1

cinemas1 <- st_set_geometry(cinemas1, "buffgeom")
cinemas2 <- st_set_geometry(cinemas2, "buffgeom")
cinemas3 <- st_set_geometry(cinemas3, "buffgeom")

cinemas_w <- rbind(cinemas1, cinemas2, cinemas3)

# Convert the buffgeom column to a spatial object
cinemas_w_sf <- st_as_sf(cinemas_w, wkt = "buffgeom")

# Create the plot data frame with the correct geometry column
plot_data <- data.frame(geometry = cinemas_w_sf$buffgeom, weight = cinemas_w_sf$weight)

# Plot the polygons with colors based on weight
ggplot() +
  geom_sf(data = plot_data, aes(geometry = geometry, fill = weight), color = "black") +
  scale_fill_gradient(low = "red", high = "green") +
  coord_sf() +
  theme_minimal()

Now we assign weights to the bikepaths/cycleways. We'll set only 2 weights - 2 for locations less than 150 m away and 1 for locations between 150 to 300m.

In [ ]:
cycleways150 <- st_buffer(cycleways_lines_UTM, dist = 150)
cycleways300 <- st_buffer(cycleways_lines_UTM, dist = 300)

cycleways150u <- st_union(cycleways150)
cycleways300u <- st_union(cycleways300)

cycleways2 <- st_sf(cycleways150u)

p1 <- ggplot() +
  geom_sf(data = cycleways2, fill = "green")

cycleways1 <- st_difference(cycleways300u,cycleways150u)
cycleways1 <- st_sf(cycleways1)

p2 <- ggplot() +
  geom_sf(data = cycleways1, fill = "red")

cycleways2$weight <- 2
cycleways1$weight <- 1

cycleways1 <- st_set_geometry(cycleways1, "buffgeom")
cycleways2 <- st_set_geometry(cycleways2, "buffgeom")

cycleways_w <- rbind(cycleways1, cycleways2)

cycleways_w_sf <- st_as_sf(cycleways_w, wkt = "buffgeom")
plot_data <- data.frame(geometry = cycleways_w_sf$buffgeom, weight = cycleways_w_sf$weight)

grid.arrange(p1, p2, ncol = 2)

ggplot() +
  geom_sf(data = plot_data, aes(geometry = geometry, fill = weight), color = "black") +
  scale_fill_gradient(low = "red", high = "green") +
  coord_sf() +
  theme_minimal()


## Intersect and sum values to find the highest value locations

In [ ]:
new_cycleways_w_sf <- cycleways_w_sf

merged_cafes <- st_union(buffered_coffee$geometry) #Union all polygons to a single Multipolygon
# run a difference overlay to take out coffeeshops
new_cycleways_w_sf$buffgeom <- sf::st_difference(cycleways_w_sf$buffgeom, merged_cafes) 

#rename column names and set geometry column
colnames(new_cycleways_w_sf) <- c('weight_cy','geometry')
new_cycleways_w_sf <- st_set_geometry(new_cycleways_w_sf, "geometry")
colnames(cinemas_w_sf) <- c('weight_ci','geometry')
cinemas_w_sf <- st_set_geometry(cinemas_w_sf, "geometry")

# run an intersect overlay operation keeping the "weight" attributes
st_agr(new_cycleways_w_sf) = "constant"
st_agr(cinemas_w_sf) = "constant"
res_union <- st_intersection(new_cycleways_w_sf, cinemas_w_sf)

#sum up the weights
res_union$final_weights <- as.integer(ifelse(is.na(res_union$weight_cy), 0, res_union$weight_cy)) +
                           as.integer(ifelse(is.na(res_union$weight_ci), 0, res_union$weight_ci))

# keep only the final_weights and geometry columns
res_union <- subset(res_union, select = c(final_weights, geometry))
colnames(res_union) <- c('weight','geometry') # rename the columns 

ggplot() +
  geom_sf(data = res_union, aes(fill = weight)) +
  theme_minimal() +
  scale_fill_gradientn(colours = rev(heat.colors(10))) +
  coord_sf() +
  labs(fill = "Weight") +
  theme(plot.title = element_text(size = 20),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 12),
        legend.text = element_text(size = 10))

Now we keep only the buildings that intersect with the map above! These buildings will be suitable for a coffee shop business!

In [ ]:
st_agr(buildings_polygons_UTM) = "constant"
st_agr(res_union) = "constant"

sites <- st_intersection(res_union, buildings_polygons_UTM)
ggplot() + geom_sf(data = sites) #Plot the result

Visualize the final result on a folium interactive map.

When you hover over each point or polygon, you’ll see the `type` and `area` of the footprint of the available building in square meters.

Feel free to play with the map by zooming in and out, it's fun!

In [ ]:
sites$geo_area <- sf::st_area(sites) # Calculate area
sites <- sf::st_transform(sites, crs = 4326) # Transform to EPSG 4326
st_agr(sites) = "constant"
sites$centroid_geom <- sf::st_centroid(sites)$geometry # Calculate centroid
coords <- st_coordinates(sites$centroid_geom)

# Get the lat and lon for centroids
sites$cent_lat <- coords[, "Y"]
sites$cent_lon <- coords[, "X"]

myIcon <- makeIcon(
  iconUrl = 'https://github.com/hourofci/hourofci-system/blob/master/hourofci-demo/supplementary/hippo-hci-tiny.png?raw=true',  
  iconWidth = 25, iconHeight = 25)

map <- leaflet(sites) %>%
  addProviderTiles(providers$Esri.NatGeoWorldMap) %>% # Add a basemap
  addMarkers(lng = ~cent_lon, lat = ~cent_lat,
             popup = ~geo_area,
            label = paste("Area: ", sites$geo_area, "  ", "Type: ", sites$building),
             icon = myIcon
            )%>% 
    addPolygons(data = sites$geometry, 
              weight = 2, 
              opacity = 1,
              color = "red", 
              dashArray = "3", 
              fillOpacity = 0.7, 
              highlightOptions = highlightOptions( 
                weight = 5,
                color = "#666",
                dashArray = "",
                fillOpacity = 0.7,
                bringToFront = TRUE), 
              label = paste("<strong>Area</strong>: ", 
                            sites$geo_area, "<br>",
                            "<strong>Type</strong>: ",
                            sites$building) %>% 
                lapply(htmltools::HTML), 
                labelOptions = labelOptions(
                style = list("font-weight" = "normal", padding = "3px 8px"),
                textsize = "15px",
                direction = "auto"))

map

# Congratulations!


**You have finished an Hour of CI!**


But, before you go ... 

1. Please fill out a very brief questionnaire to provide feedback and help us improve the Hour of CI lessons. It is fast and your feedback is very important to let us know what you learned and how we can improve the lessons in the future.
2. If you would like a certificate, then please type your name below and click "Create Certificate" and you will be presented with a PDF certificate.

<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="https://forms.gle/JUUBm76rLB8iYppN7">Take the questionnaire and provide feedback</a></font>

In [ ]:
library(magick)
library(showtext)

make_cert <- function(learner_name, lesson_name) {
  cert_filename <- "hourofci_certificate.pdf"
  
  img <- image_read("../../supplementary/hci-certificate-template.jpg")
  
  pdf_width <- 8.5  # Width of the PDF in inches 
  
  # Calculate the x-coordinate for center alignment
  learner_width <- strwidth(learner_name, units = "in", cex = 1.50)
  lesson_width <- strwidth(lesson_name, units = "in", cex = 0.8)
  
  learner_x <- (pdf_width - learner_width) / 2 
  lesson_x <- (pdf_width - lesson_width) / 2
  
  draw <- image_annotate(img, text = learner_name, size = 150,
                         color = "black", location = paste0("+", learner_x*300, "+1100"))
  draw <- image_annotate(draw, text = lesson_name, size = 80,
                         color = "black", location = paste0("+", lesson_x*300, "+1850"))
  
  image_write(draw, cert_filename, format = "pdf", density = "300")
  
  return(cert_filename)
}

# Install the 'showtext' font package
showtext_auto()

# Load the custom font
font_add("cruft", regular = "../../supplementary/cruft.ttf")

# Prompt the user for their name
name <- readline("Enter your name: ")

# Check if the name is provided
if (nzchar(name)) {
  # Input values
  lesson_name <- "Intermediate Spatial Modeling and Analytics"
  
  # Create the certificate
  filename <- make_cert(name, lesson_name)
  print(paste("Congratulations", name))
  print("Download your certificate by clicking the link below.")
  print(filename)
} else {
  print("Name not provided. Certificate creation aborted.")
}


<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="hourofci_certificate.pdf?download=1" download="hourofci_certificate.pdf">Download your certificate</a></font>